# Análise do risco de inadimplência dos mutuários

O projeto é preparar um relatório para a divisão de empréstimos de um banco. Nele eu irei descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

# Informações gerais do dataframe e imports


In [1]:
# Carregando todas as bibliotecas
import pandas as pd
import numpy as np


In [2]:
# Carregando os dados
df = pd.read_csv('credit_scoring_eng.csv')
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


## Tarefa 1. Exploração de dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo


In [3]:
# Vendo a quantidade de linhas e colunas no DF
df.shape

(21525, 12)

In [4]:
# Imprimindo as primeiras linhas do DF
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


Já a primeira vista vemos que o DF possui diversos problemas, como a presença de valores negativos e valores atipicos

In [5]:
# Obtendo informações gerais do DF
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Aparenta ter valores faltantes apenas nas colunas 'days_employed' e 'total_income'.

In [7]:
# Filtrando valores ausentes em 'days_employed'
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Vendo os dados até aqui, tiro a conclusão de que os valores faltantes são simetricos e começo a suspeitar que exista alguma relação entre elas. Investigarei mais a fundo

In [8]:
# Filtrando alguns dados nas colunas com valores ausentes
df_m_v = df[df['days_employed'].isna() & df['total_income'].isna()] 
df_m_v 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Conclusão intermediária**

O numéro de linhas na tabela filtrada corresponde exatamente ao número de valores ausentes em cada coluna. Isso indica que os valores ausentes nas duas colunas da tabela estão na mesma linha.


Aprox 1,68% dos dados estão ausentes, ou seja, a cada aproximadamente 59 dados não faltante no df existe 1 dado faltante.
Não é considerado um pedaço de extrema relevancia no df,. Ainda é cedo para afirmar qual é o motivo da coicidencia dos valores ausentes presentes nas tabelas estarem na mesma linha.



In [9]:
# quantidade de dados dentro do df
rows = df.shape[0]
columns = df.shape[1]
all_values_in_df = rows * columns

# quantidade valores faltantes dentro do df
nan_data = df['days_employed'].isna().sum() + df['total_income'].isna().sum()

#porcentagem de valores faltantes sobre os dados presentes no df
porcentage_of_missing_values = all_values_in_df/nan_data 


In [10]:
# Aqui são os dados que não possuem valores ausentes
df.dropna()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [11]:
# Verificando algumas colunas do df que possuem valores ausentes
df_m_v['education'].value_counts()
df_m_v['debt'].value_counts(normalize=True) *100
df_m_v['income_type'].value_counts()
df_m_v['dob_years'].value_counts()

dob_years
34    69
40    66
42    65
31    65
35    64
36    63
47    59
41    59
30    58
28    57
58    56
57    56
54    55
56    54
38    54
52    53
37    53
33    51
39    51
50    51
43    50
45    50
49    50
51    50
29    50
46    48
55    48
48    46
44    44
53    44
60    39
62    38
61    38
32    37
64    37
23    36
27    36
26    35
59    34
63    29
25    23
24    21
65    20
66    20
21    18
22    17
67    16
0     10
68     9
71     5
69     5
20     5
70     3
72     2
19     1
73     1
Name: count, dtype: int64



**Possíveis motivos para valores ausentes nos dados**



Feito uma breve análise apenas dos dados presentes no df é dificil dar uma resposta com certeza do porque estes dados estão faltando. Em primeira vista observando a correlação entre as cólunas 'days_employed' e 'total_income' achei que seriam pessoas nunca tivessem trabalhado antes, mas observando mais a fundo as outras colunas principalmente a 'income_type' descartei essa conclusão. O que me deixa curioso e, me impede de tender a achar que os valores faltantes são aleatorios, é essa correlação precisa em que os dados ausentes em colunas distintas estão presentes na mesma linha e correspondem a resposta da mesma pessoa.

Dito isso, a minha conclusão é: 

- É possivel que algumas pessoas não quiseram fornecer os dados
- É possivel que ocorreu algum erro coleta ou no preenchimento dos dados, o que não explica de forma satisfatoria a correlação dos dados faltantes 
- E, por fim, acredito que seja aleatorio essa presença de valores ausentes.



In [12]:
# Verificando algumas colunas novamente

df['education'].value_counts()

education
secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: count, dtype: int64

In [13]:
df['debt'].value_counts(normalize=True) *100

debt
0    91.911731
1     8.088269
Name: proportion, dtype: float64

In [14]:
df['income_type'].value_counts()

income_type
employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: count, dtype: int64

In [15]:
df['dob_years'].value_counts()

dob_years
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: count, dtype: int64

**Conclusão intermediária**


A distribuição no conjunto de dados original é semelhante a tabela filtrada com os valores ausentes. Isso pode indicar novamente que pode ter sido um erro na hora de coleta ou armazenagem dos dados.


**Conclusão intermediária**


Acredito fortemente que a ausencia desses dados foram acidentais observando apenas o df e discartando possiveis variaveis no momento do fornecimento desses dados.

**Conclusões**


O único padrão que consegui observar foi em que justamente as pessoas que tem seus dados ausentes na experiencia de trabalho, tambem tem os dados ausentes na sua renda mensal.

[Explique como você abordará os valores ausentes. Considere as categorias em que os valores estão ausentes.]

Eu pensei em duas hipoteses: Remover os dados ausentes ou substitui-los.

Remover os dados ausentes:
- Ser uma pequena porcentagem dos dados
- Preencher os dados ausentes de forma eficiente sem o uso de algum tipo de inteligencia artificial é quase que impossivel, e preencher de forma rapida com algum valor ou média dos dados pode inviezar o resultado.
- Os dados ausentes são muito importantes na inadimplência de um empréstimo, apesar de não interferir de forma tão direta  na conclusão se o estado civil de um cliente e o número de filhos 
Substuir os dados:
- É uma opção boa opção tambem

No final, eu optei substituir os dados para me forçar dominar um pouco mais o uso de algumas ferramentas do DF



## Transformação de dados


In [16]:
# Removendo duplicatas
df = df.drop_duplicates().reset_index(drop=True)


In [17]:
df['education'].value_counts()

education
secondary education    13705
bachelor's degree       4710
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        273
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: count, dtype: int64

In [18]:
df.duplicated().sum()

0

In [19]:
# Transformando os valores na coluna 'education' para low_case
df['education'] = df['education'].str.lower()


In [20]:
# Retirando o ' de bachelor's degree
df['education'] = df['education'].where(df['education'] != "bachelor's degree", 'bachelors degree')
df['education']

0           bachelors degree
1        secondary education
2        secondary education
3        secondary education
4        secondary education
                ...         
21466    secondary education
21467    secondary education
21468    secondary education
21469    secondary education
21470    secondary education
Name: education, Length: 21471, dtype: object

In [21]:
# Verificando todos os valores na coluna para ter certeza de que os corrigimos

df['education'].value_counts()

education
secondary education    15188
bachelors degree        5251
some college             744
primary education        282
graduate degree            6
Name: count, dtype: int64

In [22]:
# Vamos ver a distribuição de valores na coluna `children`
df['children'].value_counts()


children
 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: count, dtype: int64

In [23]:
# Código para verificar o porcentual de valores problematicos em 'children'
children_err = df['children'].loc[df['children'] == 20].count() + df['children'].loc[df['children'] == -1].count()
children_normal = df['children'].count() - children_err

In [24]:
children_err

123

In [25]:
children_normal

21348

Nessa coluna existem 76 pessoas que estão marcadas por terem 20 filhos, o que parece estranho. Dito isso, aparenta ser um erro de digitação onde está presente um 0 a mais nesses dados. Fora isso, 47 pessoas estão marcadas como -1 filho, o que é claramente um erro. A porcentagem dos dados aparentemente estranhos é de 0,005762% ou, pode se dizer que a cada aproximadamente 173 dados normais, existe  1 dado anormal. Dito isso, eu irei substituir a quantidade de filhos = 20 por 2, e a quantidade de filhos = -1 por 0.

In [26]:
# Corrijindo esses valores "estranhos"
df['children'].where(df['children'] != 20, 2, inplace=True )
df['children'].where(df['children'] != -1, 0, inplace=True )

In [27]:
df.query('children != 20')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelors degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21467,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21468,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21469,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [28]:
# Verificando se a coluna foi corrigida
df['children'].value_counts()

children
0    14154
1     4809
2     2128
3      330
4       41
5        9
Name: count, dtype: int64

In [29]:
# Encontrando valores problematicos em 'days_employed'
df['days_employed'].isna().sum()

2120

In [30]:
days_employed_nan = df[df['days_employed'].isna()]
days_employed = df.dropna()


x = days_employed.shape[0]
z = days_employed_nan.shape[0]

perc = x/z
perc

9.127830188679246

A cada aprox 9 dados presentes existe 1 dado faltante em 'days_employed'. Fazendo uma regra de 3, dará uma porcentagem de aprox 0,11% dos dados totais estão ausentes nessa coluna.

A coluna 'days_employed' tem diversos tipos de problema, o mais nitido é a presença de valores negativos, minha solução a eles é transforma-los em valores positivos. 

Logo em seguida me deparei com o valor em float, mas que se for transformado em numero inteiro com base na posição da virgula/ponto dá valores absurdas. Por exemplo, a pessoa com maior tempo de trabalho em dias, se convertido em anos tem mais de 1100 anos de experiencia de trabalho, o que obviamente é impossivel.

Segue abaixo a minha solução e a explicação

In [31]:

# ira retornar os valores negativos na coluna 'days_employed'
df.loc[df['days_employed'] < 0]
# retornará os valores onde a experiencia de trabalho em dias são maiores do que o tempo de vida da pessoa
df.loc[df['days_employed'] > df['dob_years'] * 365]



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
18,0,400281.136913,53,secondary education,1,widow / widower,2,F,retiree,0,9091.804,buying a second-hand car
24,1,338551.952911,57,secondary education,1,unmarried,4,F,retiree,0,46487.558,transactions with commercial real estate
25,0,363548.489348,67,secondary education,1,married,0,M,retiree,0,8818.041,buy real estate
30,1,335581.668515,62,secondary education,1,married,0,F,retiree,0,27432.971,transactions with commercial real estate
...,...,...,...,...,...,...,...,...,...,...,...,...
21451,0,338904.866406,53,secondary education,1,civil partnership,1,M,retiree,0,12070.399,to have a wedding
21454,0,386497.714078,62,secondary education,1,married,0,M,retiree,0,11622.175,property
21455,0,362161.054124,59,bachelors degree,0,married,0,M,retiree,0,11684.650,real estate transactions
21464,0,373995.710838,59,secondary education,1,married,0,F,retiree,0,24618.344,purchase of a car


In [32]:
# corrigirá os valores negativos transformandos em positivos
df['days_employed'] = df['days_employed'].abs()

In [33]:
# substituindo os valores discrepantes por nan e armazenando-os em uma nova coluna
df['days_employed_nan'] = np.where(df['dob_years'] * 365 < df['days_employed'], np.nan, df['days_employed'])
df['days_employed_nan']

0        8437.673028
1        4024.803754
2        5623.422610
3        4124.747207
4                NaN
            ...     
21466    4529.316663
21467            NaN
21468    2113.346888
21469    3112.481705
21470    1984.507589
Name: days_employed_nan, Length: 21471, dtype: float64

In [34]:
# agrupando esses valores discrepantes pela coluna 'dob_years', tirando a média e armazenando em um novo dataframe
df1 = df.groupby('dob_years').agg({'days_employed_nan': 'mean'})
df1.reset_index(inplace=True)
print(df1)

    dob_years  days_employed_nan
0           0                NaN
1          19         633.678086
2          20         684.944308
3          21         709.440930
4          22         781.376775
5          23         827.309437
6          24        1026.405485
7          25        1088.406453
8          26        1200.288052
9          27        1358.153479
10         28        1397.672853
11         29        1553.823200
12         30        1696.039355
13         31        1652.717935
14         32        1735.782175
15         33        1868.655183
16         34        1983.724384
17         35        2108.881612
18         36        2272.773915
19         37        2178.934808
20         38        2307.062965
21         39        2406.564360
22         40        2345.284329
23         41        2433.612130
24         42        2793.169884
25         43        2492.648991
26         44        2814.375145
27         45        2791.307088
28         46        2867.112262
29        

In [35]:
# renomeando a coluna no novo dataframe
df1.rename(columns = {'days_employed_nan':'avg_days_employed'}, inplace=True)
df1


,dob_years,avg_days_employed
0,0,NaN
1,19,633.678086
2,20,684.944308
3,21,709.440930
4,22,781.376775
5,23,827.309437
6,24,1026.405485
7,25,1088.406453
8,26,1200.288052
9,27,1358.153479


In [36]:
# juntando os 2 dataframes
df = df.merge(df1, on=['dob_years'], how='left')

In [37]:
# substituindo os nan pela média
df['days_employed_nan'].fillna(df['avg_days_employed'], inplace=True)

In [38]:
# fazendo uma copia de segurança antes de modificar um pouco mais a coluna 'days_employed' e antes de remover as colunas adicionadas
df_test = df.copy()
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_nan,avg_days_employed
0,1,8437.673028,42,bachelors degree,0,married,0,F,employee,0,40620.102,purchase of the house,8437.673028,2793.169884
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,4024.803754,2272.773915
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,5623.422610,1868.655183
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,4124.747207,1735.782175
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,3280.806279,3280.806279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,4529.316663,2492.648991
21467,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,3614.078059,3614.078059
21468,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,2113.346888,2307.062965
21469,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,3112.481705,2307.062965


In [39]:
# substituindo a coluna 'days_employed' pela coluna consertada e removendo as colunas adicionadas
df['days_employed'] = np.where(df['days_employed'], df['days_employed_nan'], df['days_employed'])

df.drop(columns=['days_employed_nan', 'avg_days_employed'], inplace=True)

In [40]:
# Verifique o resultado - certifique-se de que está corrigido
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelors degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,3280.806279,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21467,0,3614.078059,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21468,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21469,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [41]:
# verificando o `dob_years` para valores suspeitos e contando as porcentagens
df['dob_years'].value_counts()

dob_years
35    616
40    607
41    606
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    538
48    537
37    536
50    513
43    512
32    509
49    508
28    503
45    497
27    493
52    484
56    484
47    477
54    476
46    473
53    459
57    456
58    456
51    448
59    443
55    443
26    408
60    374
25    357
61    354
62    349
63    269
24    264
64    262
23    253
65    194
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: count, dtype: int64

Por se tratar de uma parte insignificante no dataframe, eu irei remove-los.

In [42]:
df[df['dob_years'] < 18] 
df = df[df['dob_years'] > 18]
df.reset_index(drop=True, inplace=True)

In [43]:
# Verifique o resultado - certifique-se de que está corrigido
df['dob_years'].value_counts()

dob_years
35    616
40    607
41    606
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    538
48    537
37    536
50    513
43    512
32    509
49    508
28    503
45    497
27    493
52    484
56    484
47    477
54    476
46    473
53    459
58    456
57    456
51    448
59    443
55    443
26    408
60    374
25    357
61    354
62    349
63    269
24    264
64    262
23    253
65    194
22    183
66    182
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: count, dtype: int64

In [44]:
# Verificando a coluna 'family_status'

df['family_status'].value_counts()


family_status
married              12295
civil partnership     4142
unmarried             2794
divorced              1185
widow / widower        954
Name: count, dtype: int64

In [45]:
# Vamos ver os valores na coluna 'gender'
df['gender'].value_counts()

gender
F      14117
M       7252
XNA        1
Name: count, dtype: int64

In [46]:
# abordando valores problematicos
df = df[df['gender'] != 'XNA']

In [47]:
# Verificando se está corrigido
df['gender']

0        F
1        F
2        M
3        M
4        F
        ..
21365    F
21366    F
21367    M
21368    M
21369    F
Name: gender, Length: 21369, dtype: object

In [48]:
# Vamos ver os valores na coluna 'income_type'
df['income_type'].value_counts()

income_type
employee                       11036
business                        5059
retiree                         3817
civil servant                   1451
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: count, dtype: int64

In [49]:
# Verificar duplicados

df.duplicated().sum()

17

In [50]:
# Corrijindo duplicatas
df.drop_duplicates(inplace=True)

In [51]:
# Verificando se foram removidas
df.duplicated().sum()

0

In [52]:
# Verificando o tamanho do conjunto de dados atual
df.shape

(21352, 12)

# Trabalhando com valores ausentes

In [53]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelors degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,3280.806279,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21365,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21366,0,3614.078059,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21367,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21368,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


### Restaurar valores ausentes em `total_income`

In [54]:
# Verificando mais algumas coisas
df[df['total_income'].isna()]



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,4059.315441,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,2433.612130,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,4385.872648,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,3096.417569,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,2979.965858,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21334,2,2963.394330,47,secondary education,1,married,0,M,business,0,NaN,purchase of a car
21340,1,3096.417569,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21342,0,2898.197374,48,bachelors degree,0,married,0,F,business,0,NaN,building a property
21347,1,2793.169884,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [55]:
df['dob_years'].value_counts()

dob_years
35    616
40    607
41    605
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    537
48    536
37    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    483
47    477
54    476
46    472
53    459
57    456
58    454
51    446
59    443
55    443
26    408
60    374
25    357
61    354
62    348
63    269
24    263
64    260
23    252
65    193
22    183
66    182
67    167
21    111
68     99
69     85
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: count, dtype: int64

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21352 entries, 0 to 21369
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21352 non-null  int64  
 1   days_employed     21352 non-null  float64
 2   dob_years         21352 non-null  int64  
 3   education         21352 non-null  object 
 4   education_id      21352 non-null  int64  
 5   family_status     21352 non-null  object 
 6   family_status_id  21352 non-null  int64  
 7   gender            21352 non-null  object 
 8   income_type       21352 non-null  object 
 9   debt              21352 non-null  int64  
 10  total_income      19259 non-null  float64
 11  purpose           21352 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [57]:
def age_category(data):
    
    age = data['dob_years']
    
    if  age < 30:
        return 'young'
        
    if  30 <= age < 40:
        return 'adult'
        
    if  40 <= age < 60:
        return 'middle_age'
        
    if age >= 60:
        return 'elderly'

In [58]:
# Testando a função
row_values = [1]
row_columns = ['dob_years']
row = pd.Series(data=row_values, index=row_columns)
print(age_category(row))

young


In [59]:
# Criando uma coluna usando a função

df['age_category'] = df.apply(age_category, axis=1)

In [60]:
# Verificando como ficou o df com a nova coluna

df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelors degree,0,married,0,F,employee,0,40620.102,purchase of the house,middle_age
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,3280.806279,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,middle_age
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21365,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,middle_age
21366,0,3614.078059,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,elderly
21367,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,adult
21368,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,adult


In [61]:
# Criando uma tabela com valores ausentes

df_f_nan = df.dropna()


In [62]:
df_f_nan.head(25)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelors degree,0,married,0,F,employee,0,40620.102,purchase of the house,middle_age
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,3280.806279,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,middle_age
5,0,926.185831,27,bachelors degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,young
6,0,2879.202052,43,bachelors degree,0,married,0,F,business,0,38484.156,housing transactions,middle_age
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,middle_age
8,2,6929.865299,35,bachelors degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,middle_age


In [63]:
# Vendo valores medios de 'total_income' com base em algumas caracteristicas
df_f_nan.pivot_table(index='education', columns='gender', values=['total_income','days_employed'],
                    aggfunc='mean')

days_employed               total_income              
gender                          F            M             F             M
education                                                                 
bachelors degree      2536.706074  2189.387194  30332.630701  39026.494990
graduate degree       4498.599443  2972.145010  29345.394000  27267.340000
primary education     2987.646622  2084.744344  19118.479588  23798.931664
secondary education   2885.565873  2271.187161  22677.456712  28294.556582
some college          1697.618901  1634.410465  26484.617125  33179.524066

In [64]:
# Vendo valores medianos de 'total_income' com base em algumas caracteristicas
df_pivot_median = df_f_nan.pivot_table(index=['income_type','gender'], columns='education', values='total_income',
                    aggfunc='median')
df_pivot_median

education                           bachelors degree  graduate degree  \
income_type                 gender                                      
business                    F              30352.879              NaN   
                            M              37180.918              NaN   
civil servant               F              25274.254        17822.757   
                            M              34219.850              NaN   
employee                    F              24592.436              NaN   
                            M              30812.783        31771.321   
entrepreneur                F              79866.103              NaN   
paternity / maternity leave F                    NaN              NaN   
retiree                     F              22784.685        40868.031   
                            M              25193.173        15800.399   
student                     M              15712.260              NaN   
unemployed                  F              32435.602              NaN   
                            M                    NaN              NaN   

education                           primary education  secondary education  \
income_type                 gender                                           
business                    F              21441.9210           23178.8170   
                            M              24016.1540           29102.2430   
civil servant               F              14339.0340           20133.0890   
                            M              30554.6660           27144.3130   
employee                    F              18962.6750           19872.9380   
                            M              22662.0740           24985.8485   
entrepreneur                F                     NaN                  NaN   
paternity / maternity leave F                     NaN            8612.6610   
retiree                     F              16188.8905           18046.5560   
                            M              18099.8720           20180.8795   
student                     M                     NaN                  NaN   
unemployed                  F                     NaN                  NaN   
                            M                     NaN            9593.1190   

education                           some college  
income_type                 gender                
business                    F         27275.4595  
                            M         30606.7090  
civil servant               F         21855.8610  
                            M         28371.3005  
employee                    F         21720.3035  
                            M         29977.7850  
entrepreneur                F                NaN  
paternity / maternity leave F                NaN  
retiree                     F         18654.8760  
                            M         19946.7950  
student                     M                NaN  
unemployed                  F                NaN  
                            M                NaN

In [65]:
df['education'].value_counts()

education
secondary education    15108
bachelors degree        5215
some college             741
primary education        282
graduate degree            6
Name: count, dtype: int64

In [66]:
#  Função para preencher os valores ausentes em 'total_income'
        
def salary_median(education, income_type, gender):
    try:
        return df_pivot_median[education][income_type][gender]
    except:
        return 'erro'
        

In [67]:
# Verificando se a função funciona
salary_median('bachelors degree','employee','F')

24592.436

In [68]:
# Aplicando a todas as linhas
df['median_salary'] = df.apply(lambda row: salary_median(row['education'],row['income_type'],row['gender']), axis=1)

In [69]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,median_salary
0,1,8437.673028,42,bachelors degree,0,married,0,F,employee,0,40620.102,purchase of the house,middle_age,24592.436
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult,19872.938
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult,24985.8485
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult,24985.8485
4,0,3280.806279,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,middle_age,18046.556


In [70]:
df['total_income'] = df['total_income'].fillna(df['median_salary'])

In [71]:
# Verificando o "erro"
df[df['total_income'] == 'erro']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,median_salary
5908,0,3269.12765,58,bachelors degree,0,married,0,M,entrepreneur,0,erro,buy residential real estate,middle_age,erro


In [72]:
# Substituir valores ausentes se houver algum erro
df['total_income'].replace('erro', 37180.918, inplace=True)

In [73]:
df['total_income'].dtype

dtype('float64')

In [74]:
df['total_income'].isna().sum()

0

In [75]:
# Verificar o número de entradas nas colunas

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21352 entries, 0 to 21369
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21352 non-null  int64  
 1   days_employed     21352 non-null  float64
 2   dob_years         21352 non-null  int64  
 3   education         21352 non-null  object 
 4   education_id      21352 non-null  int64  
 5   family_status     21352 non-null  object 
 6   family_status_id  21352 non-null  int64  
 7   gender            21352 non-null  object 
 8   income_type       21352 non-null  object 
 9   debt              21352 non-null  int64  
 10  total_income      21352 non-null  float64
 11  purpose           21352 non-null  object 
 12  age_category      21352 non-null  object 
 13  median_salary     21352 non-null  object 
dtypes: float64(2), int64(5), object(7)
memory usage: 2.4+ MB


###  Restaurar valores em `days_employed`

A restauração dos valores em days_employed já foi feito acima.

## Categorização de dados


In [76]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,median_salary
0,1,8437.673028,42,bachelors degree,0,married,0,F,employee,0,40620.102,purchase of the house,middle_age,24592.436
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult,19872.938
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult,24985.8485
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult,24985.8485
4,0,3280.806279,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,middle_age,18046.556


In [77]:
# Imprimindo a coluna escolhida para categorização

print(df['children'])


0        1
1        1
2        0
3        3
4        0
        ..
21365    1
21366    0
21367    1
21368    3
21369    2
Name: children, Length: 21352, dtype: int64


In [78]:
df['children'].value_counts()

children
0    14068
1     4792
2     2114
3      328
4       41
5        9
Name: count, dtype: int64

In [79]:
# Criando uma nova coluna para categorizar os dados
df['children_cate'] = df['children']

In [80]:
# Categorizando os dados
df['children_cate'].where(df['children_cate'] != 0, 'without children', inplace=True)
df['children_cate'].replace([1,2,3], 'one to three children', inplace=True)
df['children_cate'].replace([4,5], 'four or more children', inplace=True)


C:\Users\DANIEL\AppData\Local\Temp\ipykernel_5188\1599656288.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'without children' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df['children_cate'].where(df['children_cate'] != 0, 'without children', inplace=True)


In [81]:
# Obter estatísticas resumidas para a coluna

df['children_cate'].value_counts() / df['children_cate'].value_counts().sum()
# 65% das pessoas no df não possuem filhos
# 33% das pessoas no df possuem de 1 a 3 filhos
# apenas 0,02% das pessoas possuem 4 ou mais filhos

children_cate
without children         0.658861
one to three children    0.338797
four or more children    0.002342
Name: count, dtype: float64

In [82]:
df['children'].describe()

count    21352.000000
mean         0.478316
std          0.755786
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64

## Verificar as Hipóteses


**Existe uma correlação entre o nível de renda e do pagamento em dia?**

In [83]:
# Verifique os dados das crianças e do pagamento em dia

df.query('debt == 0 ')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,median_salary,children_cate
0,1,8437.673028,42,bachelors degree,0,married,0,F,employee,0,40620.102,purchase of the house,middle_age,24592.436,one to three children
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult,19872.938,one to three children
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult,24985.8485,without children
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult,24985.8485,one to three children
4,0,3280.806279,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,middle_age,18046.556,without children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21363,0,3865.467480,59,secondary education,1,married,0,F,retiree,0,24618.344,purchase of a car,middle_age,18046.556,without children
21364,1,2351.431934,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate,adult,31771.321,one to three children
21365,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,middle_age,23178.817,one to three children
21366,0,3614.078059,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,elderly,18046.556,without children


In [84]:
df.query('debt == 1 ')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,median_salary,children_cate
14,0,1844.956182,56,bachelors degree,0,civil partnership,1,F,business,1,26420.466,buy residential real estate,middle_age,30352.879,without children
32,0,4649.910832,34,secondary education,1,civil partnership,1,F,employee,1,22249.194,having a wedding,adult,19872.938,without children
38,0,597.881827,25,bachelors degree,0,unmarried,4,M,employee,1,30759.568,education,young,30812.783,without children
55,0,2979.965858,54,secondary education,1,civil partnership,1,F,retiree,1,18046.556,to have a wedding,middle_age,18046.556,without children
75,1,2953.151948,38,secondary education,1,married,0,M,employee,1,13109.661,real estate transactions,adult,24985.8485,one to three children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21339,0,960.095747,39,secondary education,1,civil partnership,1,M,business,1,28219.135,supplementary education,adult,29102.243,without children
21356,0,612.569129,29,bachelors degree,0,civil partnership,1,F,employee,1,22410.956,buying property for renting out,young,24592.436,without children
21360,1,467.685130,28,secondary education,1,married,0,F,employee,1,17517.812,to become educated,young,19872.938,one to three children
21367,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,adult,24985.8485,one to three children


In [85]:
# Calcular a taxa de inadimplência com base no número de filhos

df.query('children_cate == "four or more children"').shape[0] / df.query('debt == 1 and children_cate == "four or more children"').shape[0]
# 12.5 % das pessoas que possuem 4 ou mais filhos não pagam suas dividas

12.5

In [86]:
df.query('children_cate == "without children"').shape[0] / df.query('debt == 1 and children_cate == "without children"').shape[0]
# 13.2 % das pessoas que não possuem filhos não pagam suas dividas

13.284230406043438

In [87]:
df.query('children_cate == "one to three children"').shape[0] / df.query('debt == 1 and children_cate == "one to three children"').shape[0]
# 10.2 % das pessoas que possuem de 1 a 3 filhos não pagam suas dividas

10.797014925373134

**Conclusão**

Não existe uma correlação linear entre pessoas pessoas que não pagam suas dividas e o numero de filhos que elas possuem.


**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [88]:
# Verificando os dados de status da família e do pagamento em dia


df['family_status'].value_counts()


family_status
married              12290
civil partnership     4129
unmarried             2794
divorced              1185
widow / widower        954
Name: count, dtype: int64

In [89]:
df.pivot_table(columns='debt', index='family_status', aggfunc='count')

age_category      children      children_cate       \
debt                         0    1        0    1             0    1   
family_status                                                          
civil partnership         3743  386     3743  386          3743  386   
divorced                  1100   85     1100   85          1100   85   
married                  11363  927    11363  927         11363  927   
unmarried                 2521  273     2521  273          2521  273   
widow / widower            892   62      892   62           892   62   

                  days_employed      dob_years       ... gender       \
debt                          0    1         0    1  ...      0    1   
family_status                                        ...               
civil partnership          3743  386      3743  386  ...   3743  386   
divorced                   1100   85      1100   85  ...   1100   85   
married                   11363  927     11363  927  ...  11363  927   
unmarried                  2521  273      2521  273  ...   2521  273   
widow / widower             892   62       892   62  ...    892   62   

                  income_type      median_salary      purpose       \
debt                        0    1             0    1       0    1   
family_status                                                        
civil partnership        3743  386          3743  386    3743  386   
divorced                 1100   85          1100   85    1100   85   
married                 11363  927         11363  927   11363  927   
unmarried                2521  273          2521  273    2521  273   
widow / widower           892   62           892   62     892   62   

                  total_income       
debt                         0    1  
family_status                        
civil partnership         3743  386  
divorced                  1100   85  
married                  11363  927  
unmarried                 2521  273  
widow / widower            892   62  

[5 rows x 26 columns]

In [90]:
# Calculando a taxa padrão com base no status da família

df.query('family_status == "civil partnership"').shape[0] / df.query('debt == 1 and family_status == "civil partnership"').shape[0]

# 10.6% das pessoas com status 'civil partnership' não pagaram seu emprestimo

10.696891191709845

In [91]:
df.query('family_status == "divorced"').shape[0] / df.query('debt == 1 and family_status == "divorced"').shape[0]

# 13.9% das pessoas com status 'divorced' não pagaram seu emprestimo

13.941176470588236

In [92]:
df.query('family_status == "married"').shape[0] / df.query('debt == 1 and family_status == "married"').shape[0]

# 13.2% das pessoas com status 'married' não pagaram seu emprestimo

13.25782092772384

In [93]:
df.query('family_status == "unmarried"').shape[0] / df.query('debt == 1 and family_status == "unmarried"').shape[0]

# 10.2% das pessoas com status 'unmarried' não pagaram seu emprestimo

10.234432234432234

In [94]:
df.query('family_status == "widow / widower"').shape[0] / df.query('debt == 1 and family_status == "widow / widower"').shape[0]

# 15.3% das pessoas com status 'widow / widower' não pagaram seu emprestimo

15.387096774193548

**Conclusão**


As pessoas que mais deixam de pegar seus debitos com base no 'family_status' são:
1- widow / widower (15.3%)
2- divorced (13.9%)
3- married (13.2%)
4- civil partnership (10.6%)
5- unmarried (10.2%)

Ou seja, pessoas viuvas deixam de pagar 5% mais do que pessoas solteiras. O que, mostra que existe uma correlação entre o status civil da pessoa com a falta de pegamento de seus emprestimos/dividas.

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [95]:
# Verifique os dados do nível de renda e do pagamento em dia

df['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21365    35966.698
21366    24959.969
21367    14347.610
21368    39054.888
21369    13127.587
Name: total_income, Length: 21352, dtype: float64

In [96]:
df['debt']

0        0
1        0
2        0
3        0
4        0
        ..
21365    0
21366    0
21367    1
21368    1
21369    0
Name: debt, Length: 21352, dtype: int64

In [97]:
df.pivot_table(index='debt', values='total_income')

,total_income
debt,
0,26541.120231
1,25809.945265


In [98]:
df.pivot_table(index='debt', values='total_income', aggfunc='median')

,total_income
debt,
0,23178.817
1,22948.822


**Conclusão**


A diferença das pessoas que pagam ou não suas dividas com base em sua renda anual é extremamente baixa. Dito isso, acho que não há uma relação direta entre essas duas caracteristicas. 

**Como a finalidade do crédito afeta a taxa de inadimplência?**

In [99]:
# Confirindo os percentuais de inadimplência para cada finalidade de crédito e analisando-os

df['purpose'].value_counts().sum() 


21352

In [100]:
inad_f_c = df.query('debt == 1')['purpose'].value_counts() / df['purpose'].value_counts() 

In [101]:
inad_f_c.sort_values(ascending=False)

purpose
to get a supplementary education            0.114865
purchase of a car                           0.110132
getting higher education                    0.108491
second-hand car purchase                    0.105372
profile education                           0.101149
to own a car                                0.096842
to become educated                          0.095823
cars                                        0.092437
to buy a car                                0.092275
buying my own car                           0.091633
car purchase                                0.091503
university education                        0.089087
building a property                         0.087237
going to university                         0.087221
car                                         0.085714
getting an education                        0.083900
having a wedding                            0.083770
supplementary education                     0.082969
real estate transactions              

**Conclusão**


As pessoas que mais deixam de pagar com base no 'purpose' são as pessoas que querem o emprestimo para obter uma educação complementar. 

As pessoas que menos deixam de pagar com base no 'purpose' são as pessoas que querem o emprestimo para compra da casa própria. 

# Conclusão Geral 

As minhas conclusões finais são:

O dados possuaim diversos problemas como valores ausentes, valores atipicos, colunas categoricas com formatação não adequada e valores duplicados. Os valores ausentes eu substitui por sua media ou mediana, os valores atipicos por ser pouquissimos foram removidos, as colunas categoricas foram formatadas e os valores duplicados foram removidos.

Perguntas sobre os dados:

Existe alguma relação entre ter filhos e pagar um empréstimo em dia?
Existe alguma relação entre o estado civil e o pagamento de um empréstimo no prazo estipulado?
Existe uma relação entre o nível de renda e o pagamento de um empréstimo no prazo?
Como as diferentes finalidades do empréstimo afetam o pagamento pontual do empréstimo?

Respostas:

- Não é correto afirmar que exista uma relação entre as pessoas que deixaram de pagar seus emprestimos com a quantidade de filhos que elas possuem.
- É possivel afirmar que o estado civil implicará na relação dela pagar ou não suas dividas. Por exemplo, as pessoas nesse DF viuvas deixam de pagar os emprestimos 5% a mais do que pessoas solteiras
- É possivel afirmar que a renda anual da pessoa não alterará se ela pagará ou não suas dividas.
- É possivel afirmar que o motivo da solitação do emprestimo afetará se a pessoa pagará ou não seus emprestimos em dia.

